**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import sys
!{sys.executable} -m pip install scikit-video

     |████████████████████████████████| 2.3MB 2.7MB/s 


In [2]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd,Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization,Flatten

Using TensorFlow backend.


In [3]:
!apt-get install --no-install-recommends ffmpeg && pip install ffmpeg scikit-video

import skvideo.io
import skvideo.datasets
bbb = skvideo.datasets.bigbuckbunny()
print('bigbuckbunny is in: {}'.format(bbb))

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.6-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
  Created wheel for ffmpeg: filename=ffmpeg-1.4-cp36-none-any.whl size=6084 sha256=4149be119684bd800a4912e6328cba77bf55ff23e341c1ea04e507e17501a641
  Stored in directory: /root/.cache/pip/wheels/b6/68/c3/a05a35f647ba871e5572b9bbfc0b95fd1c6637a2219f959e7a
Successfully built ffmpeg
bigbuckbunny is in: /usr/local/lib/python3.6/dist-packages/skvideo/datasets/data/bigbuckbunny.mp4


# MiniProject on Deep Reinforcement Learning 

# Thibault LAHIRE - MVA 2019-2020

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act takes as arguments the current state ```s``` and a boolean called ```train```. It returns an action to perform from state ```s```. When ```train=False```, the function returns the action proposed by the current policy. When we know that the current policy is the best policy, we must do exploitation and set ```train``` to ```False```. However, when the agent is learning the best policy, then executing the current policy means executing a sub-optimal policy. This behaviour has two issues. First, we do not collect as much rewards as we could, and, second, the learning ends rapidly because the agent does not explore the environment (which means the agent quickly gets stuck in a local optimum).

Thus, it is a good idea to force exploration during the learning (training) phase. This can be done by using epsilon. In the proposed function, the agent acts randomly (i.e. explores) with probability ```epsilon``` and follows the current policy (i.e. exploits) with probability 1 - ```epsilon```.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the agent
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0 
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1 

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=50 # set small when debugging
epochs_test=11 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

```position``` is an array of size (grid_size, grid_size) representing the position of the agent in the environment, as suggested by its name. To the location of the agent is associated the value $+1$ in the array. To the location of the boundaries is associated the value $-1$ in the array. All others positions are assigned to the value $0$.

```board``` is also an array of size (grid_size, grid_size) containing the rewards associated to the positions. When there is cheese, the reward is equal to $0.5$. When there is poison, the reward is equal to $-1$. All the others positions are associated to the reward $0$.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, self.n_action, size=1)[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent, env, epochs, prefix=''):
    # Number of won games
    score = 0
 
    for e in range(epochs):
        # At each epoch, we start a new game
        state = env.reset()
        # This tells if the game is over
        game_over = False

        win = 0
        lose = 0
        
        
        while not game_over:
            # We take an action. train is true is the agent is learning, false otherwise.
            action = agent.act(state, train=False)
            # Collect the effects of the action
            prev_state= state
            state, reward, game_over = env.act(action)
            # Update
            if reward > 0:
                win = win + reward 
            if reward < 0:
                lose = lose - reward
                
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win - lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [10]:
# Initialize the game
env = Environment(grid_size=size, max_time=T, temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random5.mp4'))

Win/lose count 13.0/14.0. Average score (-1.0)
Win/lose count 13.0/19.0. Average score (-3.5)
Win/lose count 7.0/23.0. Average score (-7.666666666666667)
Win/lose count 7.5/17.0. Average score (-8.125)
Win/lose count 11.5/11.0. Average score (-6.4)
Win/lose count 9.5/7.0. Average score (-4.916666666666667)
Win/lose count 9.5/20.0. Average score (-5.714285714285714)
Win/lose count 7.5/11.0. Average score (-5.4375)
Win/lose count 9.5/12.0. Average score (-5.111111111111111)
Win/lose count 9.0/14.0. Average score (-5.1)
Win/lose count 13.0/16.0. Average score (-4.909090909090909)
Final score: -4.909090909090909


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




# Answer

First, we have: 

\begin{align}
Q^{\pi}(s,a) &=E \left[ \sum_{k=0}^{\infty} \gamma^k r(s_{t+k},a_{t+k})|s_t = s, a_t = a \right] \\
&=E \left[ r(s,a) + \sum_{k=1}^{\infty} \gamma^k r(s_{t+k},a_{t+k})|s_t = s, a_t = a \right] \\
&=r(s,a) + \gamma E \left[ \sum_{k=1}^{\infty} \gamma^{k-1} r(s_{t+k},a_{t+k})|s_t = s, a_t = a \right] \\
&=r(s,a) + \gamma \sum_{(s',a')} p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a ) E \left[ \sum_{k=1}^{\infty} \gamma^{k-1} r(s_{t+k},a_{t+k})|s_{t+1}=s',a_{t+1}=a' \right] \\
&=r(s,a) + \gamma \sum_{(s',a')} p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a ) E \left[ \sum_{l=0}^{\infty} \gamma^{l} r(s_{t+l+1},a_{t+l+1})|s_{t+1}=s',a_{t+1}=a' \right] \\
&=r(s,a) + \gamma \sum_{(s',a')} p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a ) Q^{\pi}(s',a') \\
&=\sum_{(s',a')} p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a ) r(s,a) + \gamma \sum_{(s',a')} p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a ) Q^{\pi}(s',a') \\
&=\sum_{(s',a')} p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a ) \left[ r(s,a) + \gamma Q^{\pi}(s',a') \right] \\
&=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{align}

Then we have:

\begin{align}
Q^{*}(s,a) &= \max_{\pi} Q^{\pi}(s',a') \\
&= \max_{\pi} \left\{ r(s,a) + \gamma \sum_{(s',a')} p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a )  Q^{\pi}(s',a')  \right\} \\
&= r(s,a) + \gamma \max_{\pi} \left\{ \sum_{(s',a')} p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a )  Q^{\pi}(s',a')  \right\} \\
&= r(s,a) + \gamma \max_{\pi} \left\{ \sum_{s'} p(s_{t+1}=s',a_{t+1}=\pi(s) |s_t = s, a_t = a )  Q^{\pi}(s',\pi(s)) \right\} \\
&= r(s,a) + \gamma \max_{\pi} \left\{ \sum_{s'} p(s_{t+1}=s' |s_t = s, a_t = a )  Q^{\pi}(s',\pi(s)) \right\} \\
\end{align}

In order to exchange the maximum and the sum, one has to remark the following.
We have the trivial inequality:

\begin{equation*}
\max_{\pi} \sum_{s'} p(s_{t+1}=s' |s_t = s, a_t = a )Q^{\pi}(s',\pi(s)) \leq \sum_{s'} p(s_{t+1}=s' |s_t = s, a_t = a ) \max_{\pi}Q^{\pi}(s',\pi(s)) 
\end{equation*}

Considering $\bar{\pi}(s') = \arg\max_{\pi} Q^{\pi}(s',\pi(s))$, we have also:
\begin{align*}
 \sum_{s'} p(s_{t+1}=s'|s_t = s, a_t = a ) \max_{\pi}Q^{\pi}(s',\pi(s)) &= \sum_{s'} p(s_{t+1}=s' |s_t = s, a_t = a ) Q^{\bar{\pi}}(s',\bar{\pi}(s)) \\
 &\leq \max_{\pi} \sum_{s'} p(s_{t+1}=s' |s_t = s, a_t = a )Q^{\pi}(s',\pi(s)) 
\end{align*}

In the following, we use the fact that, for an optimal policy $\pi^{*}$, $\pi^{*}(s) = \arg \max_{a'} Q^{*}(s,a')$.
We rewrite the initial equation as :
\begin{align}
Q^{*}(s,a)&= r(s,a) + \gamma  \sum p(s_{t+1}=s' |s_t = s, a_t = a )\max_{\pi} Q^{\pi}(s',\pi(s)) \\
&= r(s,a) + \gamma \sum(p(s_{t+1}=s' |s_t = s, a_t = a )\max_{\pi,a'} Q^{\pi}(s',a') \\ 
&= r(s,a) + \gamma \sum(p(s_{t+1}=s' |s_t = s, a_t = a )\max_{a'} Q^{*}(s',a') \\
&=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{align}

Then we define a loss:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

This loss funtion is interesting because it equals zero when $r+\gamma\max_{a'}Q(s',a',\theta) = Q(s,a,\theta)$, i.e. when the optimal Bellman equation is verified for some $\theta$.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory)>self.max_memory:
            del self.memory[0]
            

    def random_access(self):
        alea = np.random.randint(0, len(self.memory), size=1)[0]
        return self.memory[alea]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent, env, epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape([1,s.shape[0],s.shape[1],s.shape[2]]))[0,:])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
      # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, n_s_, a_, r_, game_over_  = self.memory.random_access()
            target_q[i] =self.model.predict(s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0]
            input_states[i] = s_
            if game_over_:
                target_q[i,a_] = r_
            else:
                target_q[i,a_] = r_  + self.discount*max(self.model.predict(n_s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0])
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

        
        
        
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)

        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state,)))
        model.add(Dense(64,activation ='relu')) 
        model.add(Dense(64,activation ='relu')) 
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        print(model.summary())
        self.model = model


In [14]:
epochs_train = 50 # Given the experiments I have made, this is not too much to overfit
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))





Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                3264      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 260       
Total params: 7,684
Trainable params: 7,684
Non-trainable params: 0
_________________________________________________________________
None








Epoch 000/050 | Loss 0.0040 | Win/lose count 3.5/7.0 (-3.5)
Epoch 001/050 | Loss 0.0052 | Win/lose count 3.0/2.0 (1.0)
Epoch 002/050 | Loss 0.0041 | Win/lose count 4.5/4.0 (0.5)
Epoch 003/050 | Loss 0.0104 | Win/lose count 4.5/6.0 (-

Question 7 asks us to implement a DQN with a cascade of fully connected layers. It is precised in the question that we can test different learning rates, memory sizes and batch size parameters. It is important to mention that there are many other parameters that can influence the performance of the network. Indeed, one may simply wonder before starting coding : how many layers do I use in my network ? How many neurones by layer do I use ? 

Then, questions can be asked about the optimizer. SGD is proposed, but I could use Adam, Adagrad, or RMSprop. Indeed, Adam, Adagrad and RMSprop are optimized version of SGD. However, the enhanced results of Adam, Adagrad and RMSprop are proven in the convex case, and in practice we do not have this guarantee. 

Questions can also be asked about the parameters of SGD (decay, momentum, etc...). We will now discuss the parameters that appear in question 7 : learning rate, memory size, batch size. Since it will not be possible for me to test all the possible configurations, I will rather write remarks that appear relevant to me. 

We first discuss the initial learning rate. It is probably the most important hyperparameter to tune. Indeed, an initial learning rate too high may cause a divergence of the optimization process, whereas an initial learning too low means a very long optimization process. We have tested $lr = [0.01, 0.1, 1]$. Techniques have been proposed to efficiently tune the initial learning rate. A quick search on google writing "efficiently tune initial learning rate" gives plenty of results. 

The memory size, as suggested in its name, is the size of the replay buffer in which collected samples are stored. A small memory size is a too partial view of what the agent is doing. Given the test I made, a memory size of 100 is not sufficient, whereas it is the case for a memory size of 1000.

The batch size is the number of training examples used in the estimate of the error gradient. Small batch sizes are used for two main reasons: 1) they are noisy, offering a regularizing effect and a lower generalization error and 2) they make it easier to fit one batch worth of training data in memory (i.e. when using a GPU). In practice, the default value of the batch size is often 32. A too small batch size ($<5$) has a tendency to make the learning unstable.

For this exercice, we decided to use 2 fully-connected layer of 64 units. This choice has been made given the following of this miniproject, where a 2-layer CNN is studied. In order to try to establish a "fair" comparison with the future CNN, we chose an architecture for the fully-connected that leads to a total number of parameters close to the total number of paramaters of the future CNN.

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

This question asks us to use a CNN (2 convolutional layers and one final fully connected layer). Given the shape of the map on which the agent is walking, it seems to be a good idea to use a Conv2D layer. Indeed, since the map is a grid, it seems legitimate to think that operating convolutions on grid will provide good results. 

In numerical imaging, Conv2D layers are often followed by pooling layers (e.g. VGG19). I tried several architectures but did not find any big improvements. Dropout and batch normalisation have also been tested, and I get quite bad performances. In the Reinforcement Learning community, it is a well-known fact that dropout is often a bad idea and leads to unstable results.

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(64,(2,2),input_shape=(5,5,self.n_state,),activation='relu')) 
        model.add(Conv2D(32,(2,2),activation='relu')) 
        model.add(Flatten())
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [16]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/050 | Loss 0.0007 | Win/lose count 4.0/5.0 (-1.0)
Epoch 001/050 | Loss 0.0009 | Win/lose count 1.5/5.0 (-3.5)
Epoch 002/050 | Loss 0.0033 | Win/lose count 5.0/1.0 (4.0)
Epoch 003/050 | Loss 0.0041 | Win/lose count 2.5/4.0 (-1.5)
Epoch 004/050 | Loss 0.0159 | Win/lose count 1.5/2.0 (-0.5)
Epoch 005/050 | Loss 0.0088 | Win/lose count 2.5/5.0 (-2.5)
Epoch 006/050 | Loss 0.0036 | Win/lose count 3.5/3.0 (0.5)
Epoch 007/050 | Loss 0.0268 | Win/lose count 4.0/0 (4.0)
Epoch 008/050 | Loss 0.0058 | Win/lose count 2.5/5.0 (-2.5)
Epoch 009/050 | Loss 0.0584 | Win/lose count 6.5/4.0 (2.5)
Epoch 010/050 | Loss 0.0063 | Win/lose count 2.5/4.0 (-1.5)
Epoch 011/050 | Loss 0.0043 | Win/lose count 3.0/0 (3.0)
Epoch 012/050 | Loss 0.0016 | Win/lose count 8.0/0 (8.0)
Epoch 013/050 | Loss 0.0006 | Win/lose count 4.0/0 (4.0)
Epoch 014/050 | Loss 0.0028 | Win/lose count 7.0/6.0 (1.0)
Epoch 015/050 | Loss 0.0029 | Win/lose count 7.0/3.0 (4.0)
Epoch 016/050 | Loss 0.0547 | Win/lose count 7.0/1.0 (6.0

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [17]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                3264      
_________________________________________________________________
dense_7 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_8 (Dense)              (None, 4)                 260       
Total params: 7,684
Trainable params: 7,684
Non-trainable params: 0
_________________________________________________________________
None
Test of the CNN
Win/lose count 10.0/0. Average score (10.0)
Win/lose count 1.0/0. Average score (5.5)
Win/lose count 8.5/0. Average score (6.5)
Win/lose count 8.0/0. Average score (6.875)
Win/lose count 4.5/0. Average score (6.4)
Win/lose count 

In [18]:
HTML(display_videos('cnn_test10.mp4'))

In [19]:
HTML(display_videos('fc_test10.mp4'))

When we compare the two networks, the CNN performs best. This could have been guessed during the training phase, since the CNN achieved better result than the fully connected network. We now change the temperature and we set the other parameters to the value that appears in this notebook. Note that we have kept the same training (for a temperature of 0.3) and we change the temperature only for the testing part. We obtain :

 |    Temperature    |   Final test score FC (average over several epochs)|Final test score CNN (average over several epochs)|
| :------------ | :-------------: | -------------: | 
|  0.1        | 1.5    |     2   |
|  0.2         |   2.5      |  4 |
|  0.3        |  3.5   |   5.5  |
|  0.4           | 4  |7.5      |
|  0.5         |    5  |  10.5 |
|  0.6         |   6    |  12 |
|  0.7         |  8  |  15   |
|  0.8        |   11      |    20 |
|  0.9        |   14   |   21 |

As we can see, the performance of the CNN is better than the one of the fully connected network. This gap is increasing with the temperature. Indeed, with a high temperature, the agent finds a lot of positive reward positions around him.

However, and it will be the point of question 10, the algorithm tends to not explore the map, because the agent does not have a vision of positive reward blocks far away. Hence, it does not try to explore the map deeply enough, and may go back and forth between two positions on the map.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,decay_parameter_epsilon=0.3,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we start a new game
        state = env.reset()
        
        game_over = False

        win = 0
        lose = 0
        agent.set_epsilon(agent.epsilon*(1-decay_parameter_epsilon))
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update
            if reward > 0:
                win += reward
            if reward < 0:
                lose -= reward

            # reinforce
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        # board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size)) # define maluses to get rid-off the back-and-forth behavior we have seen so far
        # coordinate of the agent
        self.x = 0
        self.y = 1

        self.t = 0

        self.scale = 16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action,train=False):
        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: not an action')

        self.t = self.t + 1     
        
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state


In [21]:
epochs_train = 50
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon =0.6, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train,decay_parameter_epsilon=0.1, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/050 | Loss 0.0194 | Win/lose count 10.5/30.000000000000117 (-19.500000000000117)
Epoch 001/050 | Loss 0.0094 | Win/lose count 8.5/20.600000000000037 (-12.100000000000037)
Epoch 002/050 | Loss 0.0299 | Win/lose count 15.0/24.100000000000048 (-9.100000000000048)
Epoch 003/050 | Loss 0.0104 | Win/lose count 16.0/18.599999999999994 (-2.5999999999999943)
Epoch 004/050 | Loss 0.0062 | Win/lose count 15.5/16.599999999999977 (-1.0999999999999766)
Epoch 005/050 | Loss 0.0492 | Win/lose count 15.0/18.699999999999974 (-3.6999999999999744)
Epoch 006/050 | Loss 0.0050 | Win/lose count 14.5/19.600000000000023 (-5.100000000000023)
Epoch 007/050 | Loss 0.0095 | Win/lose count 20.5/17.19999999999999 (3.3000000000000114)
Epoch 008/050 | Loss 0.0905 | Win/lose count 17.0/15.399999999999965 (1.6000000000000352)
Epoch 009/050 | Loss 0.0379 | Win/lose count 14.5/21.000000000000046 (-6.500000000000046)
Epoch 010/050 | Loss 0.0130 | Win/lose count 16.0/17.49999999999998 (-1.4999999999999787)
Epoch 0

In [22]:
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 20.5/3.0. Average score (17.5)
Win/lose count 27.5/3.0. Average score (21.0)
Win/lose count 20.0/2.0. Average score (20.0)
Win/lose count 22.0/5.0. Average score (19.25)
Win/lose count 25.5/1.0. Average score (20.3)
Win/lose count 22.5/3.0. Average score (20.166666666666668)
Win/lose count 25.0/4.0. Average score (20.285714285714285)
Win/lose count 22.5/3.0. Average score (20.1875)
Win/lose count 20.5/1.0. Average score (20.11111111111111)
Win/lose count 23.0/3.0. Average score (20.1)
Win/lose count 24.0/3.0. Average score (20.181818181818183)
Final score: 20.181818181818183


There are multiple ways to incorporate a decreasing epsilon-greedy exploration. For example, we can make epsilon decrease over epochs. For this exercice, we decided to make epsilon decrease over trajectories, by multiplying it with a certain decay rate (we chose 0.8) over time steps. Note that, contrarily to the previous training where epsilon was set to 0.1, we decided to set the initial epsilon to 0.9. With a temperature of 0.3, the results on training did not really improved, but we obtained much better results on testing. Indeed, the scores are about 18, which is very high !

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

Below an implementation of an agent that mimicks an expert. We obtain very good performances during the training, but mimicking an expert does not lead to being an expert ! Indeed, the scores reached by the mimicking agent are lower than those of the expert. This can be explained by the fact that the expert can create and apply new strategies to solve issues it has never met before, whereas the mimicking agent will never "innovate" and its knowledge is necessarily inferior to the knowledge of the expert.

In [0]:
class DQN_Mim(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN_Mim, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape([1,s.shape[0],s.shape[1],s.shape[2]]))[0,:])

    def reinforce(self, s_, a_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_[:,:,1:], a_]) #now our model dirctly copy what the expert has done 
        #therefore we juste need the current state and the action that has been applied
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, a_ = self.memory.random_access()
            input_states[i] = s_
            target_q[i,a_] = 5  # mimick the expert
            
        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model
            
class DQN_CNN_Mim(DQN_Mim):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN_Mim, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(64,(2,2),input_shape=(5,5,self.n_state,),activation='relu')) 
        model.add(Conv2D(32,(2,2),activation='relu')) 
        model.add(Flatten())
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [0]:
def train_mimick(expert,agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0    
    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        while not game_over:
            # The agent performs an action
            action = expert.act(state,train=False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state,  action)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [25]:
epochs_train = 50
expert = agent
agent_n = DQN_CNN_Mim(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=2)
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
train_mimick(expert,agent_n, env, epochs_train, prefix='cnn_train_mimick')

Epoch 000/050 | Loss 1.1726 | Win/lose count 27.5/1.0 (26.5)
Epoch 001/050 | Loss 1.4660 | Win/lose count 23.5/1.0 (22.5)
Epoch 002/050 | Loss 1.9936 | Win/lose count 22.0/3.0 (19.0)
Epoch 003/050 | Loss 1.9727 | Win/lose count 19.5/2.0 (17.5)
Epoch 004/050 | Loss 1.6824 | Win/lose count 24.5/3.0 (21.5)
Epoch 005/050 | Loss 1.0875 | Win/lose count 20.5/2.0 (18.5)
Epoch 006/050 | Loss 1.3968 | Win/lose count 23.5/1.0 (22.5)
Epoch 007/050 | Loss 1.1951 | Win/lose count 23.5/3.0 (20.5)
Epoch 008/050 | Loss 1.3048 | Win/lose count 26.0/3.0 (23.0)
Epoch 009/050 | Loss 1.1559 | Win/lose count 21.5/1.0 (20.5)
Epoch 010/050 | Loss 1.5166 | Win/lose count 22.0/3.0 (19.0)
Epoch 011/050 | Loss 0.9529 | Win/lose count 24.0/2.0 (22.0)
Epoch 012/050 | Loss 1.6256 | Win/lose count 24.5/5.0 (19.5)
Epoch 013/050 | Loss 1.3892 | Win/lose count 12.5/2.0 (10.5)
Epoch 014/050 | Loss 1.0913 | Win/lose count 23.0/2.0 (21.0)
Epoch 015/050 | Loss 1.9219 | Win/lose count 22.0/2.0 (20.0)
Epoch 016/050 | Loss 1.4

In [26]:
env_test = Environment(grid_size=size, max_time=T,temperature=0.3)
test(agent_n,env_test,epochs_test,prefix='cnn_test_mimick')

Win/lose count 17.5/4.0. Average score (13.5)
Win/lose count 9.5/1.0. Average score (11.0)
Win/lose count 14.0/2.0. Average score (11.333333333333334)
Win/lose count 17.5/3.0. Average score (12.125)
Win/lose count 16.5/3.0. Average score (12.4)
Win/lose count 13.5/0. Average score (12.583333333333334)
Win/lose count 3.0/0. Average score (11.214285714285714)
Win/lose count 14.5/0. Average score (11.625)
Win/lose count 8.0/1.0. Average score (11.11111111111111)
Win/lose count 16.0/2.0. Average score (11.4)
Win/lose count 22.0/4.0. Average score (12.0)
Final score: 12.0


In [27]:
HTML(display_videos('cnn_test_mimick10.mp4'))

The fact that mimicking an expert doesn't mean being an expert can be seen during the test phase (it highly depends on the experiment). Indeed, if good results were obtained during the training, this agent is not able to generalize its knowledge and is sometimes lost during the test phase.